# Engineering Notebook

## By Rakesh Nori, Soham Bose
This is the engineering notebook for the third design project in CS 262. Our assignment was to re-design the gRPC chat application we made in our first design project so that it is both persistent(meaning that a server can be stopped and reinitialized back to the most recent primary server state) and 2-fault tolerant(meaning that the client experience will not be affected even if 2 servers fail). 

## Replication

### Primary / Secondary Model:

For this project, we chose to go with a simple primary / secondary replication model. Under this schema, the client interacts with the primary server for all of its requests, and the primary server relays all write requests (e.g. create account, delete account, login, logout, send message) to the secondary servers so that they can have the same state as the primary at all times. 

To accomplish the primary / secondary model, we need to implement three components: leader election, relaying writes, and connecting the client to the primary server. The sections below will go over how we implemented these components to accomplish replication. 

### Leader Election

Under our model, we hard coded the server addresses and gave them a fixed order. To implement the fixed order, we mapped each server address to an id, and the server that has the lowest id and running will be the primary server. 

Leader election is done asynchronously in a separate thread under the method ElectLeader(), which is ran once every second. In this method, a server asks for the ids of all servers and keeps track of the minimum id received from a running server. Once it completes its queries to all servers, it elects the server with the minimum id reported to be the primary leader and tells all of the other servers to do the same. 

We added two new RPC calls for leader election: GetServerId and UpdatePrimaryServer. The first retrieves the id associated with a server, and the second tells updates another server with the id of the new primary server. Both are used in the leader election algorithm.

### Relaying Actions

Now that we have a way to elect our primary server, we need to make sure that it can correctly handle RPC requests and ensure that the secondary servers mimic the correct ones. For our chat service, we need secondary servers to be aware of account creations, deletions, logins, logouts, message sends, and the messages that the user has not yet seen such that the failure of a primary server can enable a seamless transition to the secondary server without the client noticing anything underneath the hood. 

To accomplish this, we modified one RPC call for relaying write actions (UpdateChatLength) and modified all of the previous write RPC requests to include a new field called “fromPrimary.” This field helps a server determine whether the request was given to it from a client or the primary server. Through this attribute, we can make secondary servers answer RPC calls only from the primary server and have our primary server be able to relay the write actions it does to the secondary servers. 

We then modified all of the RPC methods to have the server execute it only if they are the primary server or if they are receiving the call from the primary server. Furthermore, If the primary server does a call, it must relay the request to the other servers and set the fromPrimary flag to be true. 

### Client Side Changes

Lastly, we must ensure that the client is always connected to the primary server. We added one RPC call to help connect the client to the primary server: GetPrimaryServerId. Then, we implemented a method called connectPrimary, which is run in a separate thread from the client in a loop and ensures that the connection between the client and server is always to the primary server. In that method, we constantly check for primary server crashes. When it crashes, we make the client connect to another server, use getPrimaryServer() to fetch the new primary server, and ultimately connect the client to that primary server. 

## Persistence

### Choosing Actions to Log

We decided to implement server persistence as defined above so that a server is able to reinitialize at the most recent state of the primary server in case any of the servers crash. In order to do this, we kept a log of all requisite actions that would be necessary for the server to go through during the process of reinitialization. The actions we decided to log are CreateAccount, DeleteAccount, Login, Logout, SendMessage, UpdateChatLength. These actions would all lead to an update to the state of the server, with CreateAccount and DeleteAccount changing the length and contents of the users set, Login and Logout changing the length and contents of the online set, and SendMessage and UpdateChatLength changing the length and contents of the chat and chat_lens dictionaries. Since all of these actions cause a change in the state of the server, the lack of replication of these actions would lead to conflicting states from the primary server at any moment, which would lead to a divergence in the states. In comparison, actions from the client such as ListAccounts don't cause a change in the state of the server as it is just a listing of the current accounts and doesn't cause a change in the associated objects of the server. 

### Logging Process

For each of the actions that we chose to log, we stored the requisite information necessary to make client calls to the server methods, the keyword arguments that were passed to each of the methods. Specifically, for CreateAccount, DeleteAccount, Login, and Logout, only the username that was passed as a part of the original request along with the context was stored. For SendMessage, the sender, recipient, and actual message was stored, the necessary keywords necessary to recreate a SendMessage method call from a client. For UpdateChatLength, when it is called in the ChatStream method, the account name and the current chat length of the user is stored in the csv. A main condition that we imposed is that only the primary elected server could add to the log. This allowed us to keep track of actions that were made by the primary server which could then be recreated in any other server that is necessary to be spun up in the case of the primary server crashing or all servers going down. 

### Recreating Logged Actions

In order to recreate the logged actions, we went through the csv row by row when initializing any server. The specific logging csv is created if it doesn't locally exist and looped through if it does locally exist. Since each row contains the function name that was called from the server, we create a map to allow us to know which requests would be made to a server by a client. Thus, during initialization, we made calls to server methods as if they were made from a client using the provided information in the csv - the keyword arguments and the map to the request call along with the context. This allowed us to recreate the final state of the primary elected server at the time that it went down or the processes related to all servers were shut down. 